In [1]:
import mlflow 
import re 
import string 
import joblib
import numpy as np
import pandas as pd 
import mlflow.sklearn
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/refs/heads/main/tweet_emotions.csv")
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [3]:
df.drop(columns=['tweet_id'], inplace=True)

In [4]:
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
x = df['sentiment'].isin(['happiness' , 'sadness'])
df = df[x]
df.head()

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?


In [6]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)


def remove_stop_words(text):
    stop_words = set(stopwords.words('english'))

    text = [word for word in str(text).split() if word not in stop_words]
    
    return " ".join(text)

def removing_numbers(text):

    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text


def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


def normalize_text(df):

    try: 
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df

    except Exception as e: 
        print(f'Error during text normalization: {e}')
        raise        

<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
C:\Users\vinay\AppData\Local\Temp\ipykernel_7952\1399214902.py:29: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [7]:
df = normalize_text(df)
df.head()

,sentiment,content
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
6,sadness,sleep im not thinking old friend want married ...
8,sadness,charviray charlene love miss
9,sadness,kelcouch sorry least friday


In [8]:
df.sentiment.value_counts()

sentiment
happiness    5209
sadness      5165
Name: count, dtype: int64

In [9]:
df['sentiment'].replace({'sadness':0 , 'happiness':1}, inplace=True)

C:\Users\vinay\AppData\Local\Temp\ipykernel_7952\2069800910.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].replace({'sadness':0 , 'happiness':1}, inplace=True)
C:\Users\vinay\AppData\Local\Temp\ipykernel_7952\2069800910.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sen

In [10]:
df.sentiment.value_counts()

sentiment
1    5209
0    5165
Name: count, dtype: int64

In [11]:
# bow 

vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [12]:
X.shape , y.shape

((10374, 1000), (10374,))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
import dagshub

dagshub.init(repo_owner='vinayak910', repo_name='mlops-mini', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/vinayak910/mlops-mini.mlflow")

mlflow.set_experiment("Logistic Regression as Baseline")

Accessing as vinayak910

Initialized MLflow to track repo "vinayak910/mlops-mini"

Repository vinayak910/mlops-mini initialized!

2025/06/25 07:27:37 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression as Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/70329c649c4c451a8f0036637ecc62a7', creation_time=1750816658679, experiment_id='0', last_update_time=1750816658679, lifecycle_stage='active', name='Logistic Regression as Baseline', tags={}>

In [15]:
with mlflow.start_run():

    mlflow.log_param('vectorizer', 'Bag of Words')
    mlflow.log_param('num_features' , 1000)
    mlflow.log_param('test_size' , 0.2)

    model = LogisticRegression()
    model.fit(X_train , y_train)

    mlflow.log_param('model' , 'Logistic Regression')

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test , y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test , y_pred) 

    mlflow.log_metric("accuracy" , accuracy)
    mlflow.log_metric("precision" , precision)
    mlflow.log_metric("recall" , recall)
    mlflow.log_metric("f1 score" , f1)

    import os 
    os.system(f"jupyter nbconvert --to notebook --execute --inplace exp1_baseline.ipynb")

    print(f"Accuracy:{accuracy}")
    print(f"Recall: {recall}")
    print(f"Precision:{precision}")
    print(f"f1_score: {f1_score}")



Accuracy:0.7749397590361445
Recall: 0.7773399014778325
Precision:0.7660194174757281
f1_score: <function f1_score at 0x000001F84BE1EAC0>
🏃 View run bald-roo-698 at: https://dagshub.com/vinayak910/mlops-mini.mlflow/#/experiments/0/runs/8ba2425dff3d422fa90c6126c539294a
🧪 View experiment at: https://dagshub.com/vinayak910/mlops-mini.mlflow/#/experiments/0
